In [1]:
pip install tensorflow keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, Concatenate, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


2024-06-18 02:25:47.614933: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 02:25:47.615068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 02:25:47.756219: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def conv_block(x, filters, kernel_size, strides=(1, 1), padding='same', use_relu=True):
    x = Conv2D(filters, kernel_size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    if use_relu:
        x = Activation('relu')(x)
    return x

def inception_stem(input_tensor):
    # Define the stem block of the Inception-v4 architecture here
    # This is a simplified version for demonstration
    x = conv_block(input_tensor, 32, (3, 3), strides=(2, 2), padding='valid')
    x = conv_block(x, 32, (3, 3), padding='valid')
    x = conv_block(x, 64, (3, 3))
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(x)
    return x

def inception_a(input_tensor):
    # Define the Inception-A block here
    # This is a simplified version for demonstration
    branch1x1 = conv_block(input_tensor, 64, (1, 1))
    
    branch5x5 = conv_block(input_tensor, 48, (1, 1))
    branch5x5 = conv_block(branch5x5, 64, (5, 5))
    
    branch3x3dbl = conv_block(input_tensor, 64, (1, 1))
    branch3x3dbl = conv_block(branch3x3dbl, 96, (3, 3))
    branch3x3dbl = conv_block(branch3x3dbl, 96, (3, 3))
    
    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(input_tensor)
    branch_pool = conv_block(branch_pool, 32, (1, 1))
    
    x = Concatenate()([branch1x1, branch5x5, branch3x3dbl, branch_pool])
    return x

def reduction_a(input_tensor):
    # Define the Reduction-A block here
    # This is a simplified version for demonstration
    branch3x3 = conv_block(input_tensor, 384, (3, 3), strides=(2, 2), padding='valid')
    
    branch3x3dbl = conv_block(input_tensor, 64, (1, 1))
    branch3x3dbl = conv_block(branch3x3dbl, 96, (3, 3))
    branch3x3dbl = conv_block(branch3x3dbl, 96, (3, 3), strides=(2, 2), padding='valid')
    
    branch_pool = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(input_tensor)
    
    x = Concatenate()([branch3x3, branch3x3dbl, branch_pool])
    return x

# Additional inception and reduction blocks can be similarly defined (inception_b, reduction_b, inception_c)


In [5]:
def inception_b(input_tensor):
    branch1x1 = conv_block(input_tensor, 384, (1, 1))
    
    branch7x7 = conv_block(input_tensor, 192, (1, 1))
    branch7x7 = conv_block(branch7x7, 224, (1, 7))
    branch7x7 = conv_block(branch7x7, 256, (7, 1))
    
    branch7x7dbl = conv_block(input_tensor, 192, (1, 1))
    branch7x7dbl = conv_block(branch7x7dbl, 192, (7, 1))
    branch7x7dbl = conv_block(branch7x7dbl, 224, (1, 7))
    branch7x7dbl = conv_block(branch7x7dbl, 224, (7, 1))
    branch7x7dbl = conv_block(branch7x7dbl, 256, (1, 7))
    
    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(input_tensor)
    branch_pool = conv_block(branch_pool, 128, (1, 1))
    
    x = Concatenate()([branch1x1, branch7x7, branch7x7dbl, branch_pool])
    return x

def reduction_b(input_tensor):
    branch3x3 = conv_block(input_tensor, 192, (1, 1))
    branch3x3 = conv_block(branch3x3, 192, (3, 3), strides=(2, 2), padding='valid')
    
    branch7x7x3 = conv_block(input_tensor, 256, (1, 1))
    branch7x7x3 = conv_block(branch7x7x3, 256, (1, 7))
    branch7x7x3 = conv_block(branch7x7x3, 320, (7, 1))
    branch7x7x3 = conv_block(branch7x7x3, 320, (3, 3), strides=(2, 2), padding='valid')
    
    branch_pool = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(input_tensor)
    
    x = Concatenate()([branch3x3, branch7x7x3, branch_pool])
    return x

def inception_c(input_tensor):
    branch1x1 = conv_block(input_tensor, 256, (1, 1))
    
    branch3x3 = conv_block(input_tensor, 384, (1, 1))
    branch3x3_1 = conv_block(branch3x3, 256, (1, 3))
    branch3x3_2 = conv_block(branch3x3, 256, (3, 1))
    branch3x3 = Concatenate()([branch3x3_1, branch3x3_2])
    
    branch3x3dbl = conv_block(input_tensor, 384, (1, 1))
    branch3x3dbl = conv_block(branch3x3dbl, 448, (1, 3))
    branch3x3dbl = conv_block(branch3x3dbl, 512, (3, 1))
    branch3x3dbl_1 = conv_block(branch3x3dbl, 256, (1, 3))
    branch3x3dbl_2 = conv_block(branch3x3dbl, 256, (3, 1))
    branch3x3dbl = Concatenate()([branch3x3dbl_1, branch3x3dbl_2])
    
    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(input_tensor)
    branch_pool = conv_block(branch_pool, 256, (1, 1))
    
    x = Concatenate()([branch1x1, branch3x3, branch3x3dbl, branch_pool])
    return x


In [6]:
def inception_v4(input_shape, num_classes):
    input_tensor = Input(shape=input_shape)
    
    # Stem block
    x = inception_stem(input_tensor)
    
    # Inception-A blocks
    for _ in range(4):
        x = inception_a(x)
    
    # Reduction-A block
    x = reduction_a(x)
    
    # Inception-B blocks
    for _ in range(7):
        x = inception_b(x)
    
    # Reduction-B block
    x = reduction_b(x)
    
    # Inception-C blocks
    for _ in range(3):
        x = inception_c(x)
    
    # Final layers
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)
    
    model = Model(input_tensor, output_tensor)
    return model


In [8]:
input_shape = (128, 128, 1)
num_classes = 2
model = inception_v4(input_shape, num_classes)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 63, 63, 32)           320       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 63, 63, 32)           128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 63, 63, 32)           0         ['batch_normalization[0][0

In [1]:
pip install tensorflow keras scipy scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, Concatenate, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from scipy.sparse.linalg import eigsh
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img


2024-06-18 03:29:19.907298: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 03:29:19.907420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 03:29:20.021320: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def locality_preserving_projection(X, n_neighbors=5, n_components=30):
    neighbors = NearestNeighbors(n_neighbors=n_neighbors).fit(X)
    distances, indices = neighbors.kneighbors(X)
    
    W = np.zeros((X.shape[0], X.shape[0]))
    for i in range(X.shape[0]):
        for j in range(n_neighbors):
            W[i, indices[i, j]] = np.exp(-distances[i, j] ** 2)
            W[indices[i, j], i] = W[i, indices[i, j]]
    
    D = np.diag(W.sum(axis=1))
    L = D - W
    
    D_inv = np.linalg.inv(D)
    XDXT = X.T @ D @ X
    XLXT = X.T @ L @ X
    
    _, eigvecs = eigsh(XLXT, k=n_components, M=XDXT, which='SM')
    
    return eigvecs


In [5]:
data_dir = "/kaggle/input/breastcancer1/dataset"
categories = ['Benign', 'Malignant']

def load_data(data_dir, categories, img_size=(128, 128)):
    data = []
    labels = []
    
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
        
        for img in os.listdir(path):
            try:
                img_path = os.path.join(path, img)
                img = load_img(img_path, color_mode='grayscale', target_size=img_size)
                img = img_to_array(img)
                data.append(img)
                labels.append(class_num)
            except Exception as e:
                pass
    
    data = np.array(data, dtype='float32')
    labels = np.array(labels, dtype='int')
    
    return data, labels

img_size = (128, 128)
data, labels = load_data(data_dir, categories, img_size)

data = data / 255.0
labels = tf.keras.utils.to_categorical(labels, num_classes=2)

# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat = X_val.reshape(X_val.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Apply LPP on the training data
lpp_features_train = locality_preserving_projection(X_train_flat, n_components=30)

# Apply the learned LPP transformation to the validation and test data
lpp_features_val = X_val_flat @ lpp_features_train# Apply LPP
lpp_projection_matrix = locality_preserving_projection(X_train_flat, n_components=30)
lpp_features_train = X_train_flat @ lpp_projection_matrix
lpp_features_val = X_val_flat @ lpp_projection_matrix
lpp_features_test = X_test_flat @ lpp_projection_matrix

# Ensure shapes are consistent
print(X_train.shape, lpp_features_train.shape, y_train.shape)
print(X_val.shape, lpp_features_val.shape, y_val.shape)
print(X_test.shape, lpp_features_test.shape, y_test.shape)

In [7]:
def conv_block(x, filters, kernel_size, strides=(1, 1), padding='same', use_relu=True):
    x = Conv2D(filters, kernel_size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    if use_relu:
        x = Activation('relu')(x)
    return x

def inception_stem(input_tensor):
    x = conv_block(input_tensor, 32, (3, 3), strides=(2, 2), padding='valid')
    x = conv_block(x, 32, (3, 3), padding='valid')
    x = conv_block(x, 64, (3, 3))
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(x)
    return x

def inception_a(input_tensor):
    branch1x1 = conv_block(input_tensor, 64, (1, 1))
    branch5x5 = conv_block(input_tensor, 48, (1, 1))
    branch5x5 = conv_block(branch5x5, 64, (5, 5))
    branch3x3dbl = conv_block(input_tensor, 64, (1, 1))
    branch3x3dbl = conv_block(branch3x3dbl, 96, (3, 3))
    branch3x3dbl = conv_block(branch3x3dbl, 96, (3, 3))
    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(input_tensor)
    branch_pool = conv_block(branch_pool, 32, (1, 1))
    x = Concatenate()([branch1x1, branch5x5, branch3x3dbl, branch_pool])
    return x

def reduction_a(input_tensor):
    branch3x3 = conv_block(input_tensor, 384, (3, 3), strides=(2, 2), padding='valid')
    branch3x3dbl = conv_block(input_tensor, 64, (1, 1))
    branch3x3dbl = conv_block(branch3x3dbl, 96, (3, 3))
    branch3x3dbl = conv_block(branch3x3dbl, 96, (3, 3), strides=(2, 2), padding='valid')
    branch_pool = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(input_tensor)
    x = Concatenate()([branch3x3, branch3x3dbl, branch_pool])
    return x

def inception_b(input_tensor):
    branch1x1 = conv_block(input_tensor, 384, (1, 1))
    branch7x7 = conv_block(input_tensor, 192, (1, 1))
    branch7x7 = conv_block(branch7x7, 224, (1, 7))
    branch7x7 = conv_block(branch7x7, 256, (7, 1))
    branch7x7dbl = conv_block(input_tensor, 192, (1, 1))
    branch7x7dbl = conv_block(branch7x7dbl, 192, (7, 1))
    branch7x7dbl = conv_block(branch7x7dbl, 224, (1, 7))
    branch7x7dbl = conv_block(branch7x7dbl, 224, (7, 1))
    branch7x7dbl = conv_block(branch7x7dbl, 256, (1, 7))
    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(input_tensor)
    branch_pool = conv_block(branch_pool, 128, (1, 1))
    x = Concatenate()([branch1x1, branch7x7, branch7x7dbl, branch_pool])
    return x

def reduction_b(input_tensor):
    branch3x3 = conv_block(input_tensor, 192, (1, 1))
    branch3x3 = conv_block(branch3x3, 192, (3, 3), strides=(2, 2), padding='valid')
    branch7x7x3 = conv_block(input_tensor, 256, (1, 1))
    branch7x7x3 = conv_block(branch7x7x3, 256, (1, 7))
    branch7x7x3 = conv_block(branch7x7x3, 320, (7, 1))
    branch7x7x3 = conv_block(branch7x7x3, 320, (3, 3), strides=(2, 2), padding='valid')
    branch_pool = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(input_tensor)
    x = Concatenate()([branch3x3, branch7x7x3, branch_pool])
    return x

def inception_c(input_tensor):
    branch1x1 = conv_block(input_tensor, 256, (1, 1))
    branch3x3 = conv_block(input_tensor, 384, (1, 1))
    branch3x3_1 = conv_block(branch3x3, 256, (1, 3))
    branch3x3_2 = conv_block(branch3x3, 256, (3, 1))
    branch3x3 = Concatenate()([branch3x3_1, branch3x3_2])
    branch3x3dbl = conv_block(input_tensor, 384, (1, 1))
    branch3x3dbl = conv_block(branch3x3dbl, 448, (1, 3))
    branch3x3dbl = conv_block(branch3x3dbl, 512, (3, 1))
    branch3x3dbl_1 = conv_block(branch3x3dbl, 256, (1, 3))
    branch3x3dbl_2 = conv_block(branch3x3dbl, 256, (3, 1))
    branch3x3dbl = Concatenate()([branch3x3dbl_1, branch3x3dbl_2])
    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(input_tensor)
    branch_pool = conv_block(branch_pool, 256, (1, 1))
    x = Concatenate()([branch1x1, branch3x3, branch3x3dbl, branch_pool])
    return x

def inception_v4_with_lpp(input_shape, lpp_shape, num_classes):
    input_tensor = Input(shape=input_shape)
    lpp_input = Input(shape=lpp_shape)
    
    x = inception_stem(input_tensor)
    for _ in range(4):
        x = inception_a(x)
    x = reduction_a(x)
    for _ in range(7):
        x = inception_b(x)
    x = reduction_b(x)
    for _ in range(3):
        x = inception_c(x)
    
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    
    combined = Concatenate()([x, lpp_input])
    
    output_tensor = Dense(num_classes, activation='softmax')(combined)
    
    model = Model([input_tensor, lpp_input], output_tensor)
    return model


In [ ]:
input_shape = (128, 128, 1)
lpp_shape = (lpp_features_train.shape[1],)
num_classes = 2

model = inception_v4_with_lpp(input_shape, lpp_shape, num_classes)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


In [ ]:
# Callbacks for saving the best model and early stopping
#checkpoint = ModelCheckpoint('inception_v4_lpp_best_model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

# Training the model
history = model.fit(
    [X_train, lpp_features_train],
    y_train,
    batch_size=32,
    epochs=10,
    validation_data=([X_val, lpp_features_val], y_val),
    callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate([X_test, lpp_features_test], y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Predict on the test set
y_pred_probs = model.predict([X_test, lpp_features_test])
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate accuracy, precision, and recall
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")


In [ ]:
# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plotting the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Plot the learning curves
plt.figure(figsize=(12, 8))

# Accuracy
plt.subplot(2, 1, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

# Loss
plt.subplot(2, 1, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

plt.tight_layout()
plt.show()